<a href="https://colab.research.google.com/github/dldmstj0531/GEC/blob/main/notebooks/model/baseline_T5%2BLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 1.1: 라이브러리 설치
!pip install -q datasets transformers[torch] peft evaluate sacrebleu

In [ ]:
# 1.2: 라이브러리 임포트
import os
import torch
import numpy as np
import evaluate # 평가 지표 로드
from datasets import load_dataset, DatasetDict
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
# 1.3: 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

## 2. 모델 및 LoRA 설정

In [ ]:
# 2.1: T5 모델 및 토크나이저 로드
MODEL_NAME = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
# 2.2: LoRA 설정 (PEFT)
# T5는 Seq2Seq 모델이므로 task_type을 명시.
config = LoraConfig(
    r=16,       # 8 or 16
    lora_alpha=32,      # r * 2
    target_modules=["q", "v"],      # T5 경우 보통 `q`, `v` 레이어에 적용
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # T5 = SEQ_2_SEQ_LM
)

In [ ]:
# 2.3: 모델에 LoRA 적용
peft_model = get_peft_model(model, config)
# peft_model

# 2.4: 학습 가능한 파라미터 확인
print("--- LoRA 적용 후 학습 파라미터 ---")
peft_model.print_trainable_parameters()

## 3. 데이터 로드 및 전처리

In [ ]:
# 3.1: CSV 파일에서 데이터셋 로드
TRAIN_PATH = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/c4_200m.csv"

raw_train_dataset = load_dataset("csv", data_files={"train": TRAIN_PATH}, split="train")

In [ ]:
# 3.2: 1만 개 랜덤 샘플링
# (seed=42는 재현성을 위함. 변경 가능)
if len(raw_train_dataset) > 10000:
    sampled_train_dataset = raw_train_dataset.shuffle(seed=42).select(range(10000))
else:
    sampled_train_dataset = raw_train_dataset.shuffle(seed=42) # 1만개 이하면 그냥 섞기만 함

print(f"--- 원본 데이터 {len(raw_train_dataset)}개에서 {len(sampled_train_dataset)}개 샘플링 ---")

In [ ]:
# 3.3: 샘플링된 5만 개를 훈련(train)과 검증(validation) 세트로 분할
#       : 90% 훈련(45k), 10% 검증(5k)
train_val_split = sampled_train_dataset.train_test_split(test_size=0.1)

# 최종 데이터셋 구성 (이제 "test" 세트는 여기에 포함되지 않음)
datasets = DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
})

print("--- 훈련/검증 데이터셋 구성 ---")
print(datasets)

In [ ]:
# 3.4: T5 접두사(Prefix) 및 전처리 함수 정의
PREFIX = "grammar correction: "
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128

def preprocess_function(examples):
    # 입력 (Noise): "grammar correction: [오류 문장]"
    inputs = [PREFIX + doc for doc in examples["noise"]]

    # 타겟 (Clean): "[수정된 문장]"
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        # padding="max_length"
    )

    # 타겟(레이블) 토크나이징
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["clean"],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            # padding="max_length"
        )

    # T5는 패딩된 레이블을 -100으로 설정하여 손실 계산에서 제외
    # labels["input_ids"] = [
    #     [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    # ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# 3.5: 전처리 함수 적용 (데이터셋 전체에 매핑)
tokenized_datasets = datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=datasets["train"].column_names
)
print(tokenized_datasets)

In [ ]:
# 3.6: 데이터 콜레이터 정의
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=peft_model
)

## 4. 평가 지표 (Metrics) 정의

- "validation" 세트를 평가할 때 사용

In [ ]:
# 4.1: 평가 지표 로드 (sacreBLEU)
sacrebleu = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # 예측 결과(preds) 디코딩
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # 레이블(labels) 디코딩. (-100은 패딩이므로 무시)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # sacreBLEU 계산을 위해 후처리 (공백 제거)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    # sacreBLEU는 [참조] 형식이 아닌 [[참조]] 형식을 요구
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": result["score"]}

## 5. TrainingArguments 및 Trainer 정의

In [ ]:
# 5.1: TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,

    # 배치 사이즈를 8 -> 4 -> 2로 줄여서 메모리 사용량 감소
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,  # eval 배치도 함께 줄이기.

    # 줄어든 배치 사이즈를 보완 (4 * 2 = 8, 기존 배치와 동일한 효과)
    gradient_accumulation_steps=4,

    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,

    eval_strategy="epoch",            # 매 에포크마다 검증
    save_strategy="epoch",            # 매 에포크마다 모델 저장
    load_best_model_at_end=True,      # 학습 종료 시 최고 성능 모델 로드

    # metric_for_best_model="bleu",     # "bleu"를 기준으로 최고 모델 저장
    # greater_is_better=False,          # bleu 점수는 높을수록 좋음

    metric_for_best_model="loss",     # "loss"를 기준으로 최고 모델 저장
    greater_is_better=False,          # Loss는 낮을수록 좋음

    fp16=True,                        # (GPU 사용 시) 혼합 정밀도 학습
    report_to="none",
)

In [ ]:
# 5.2: Trainer 정의
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,

    # [수정] 이 라인을 주석 처리/삭제하여 평가 시 .generate()를 막음
    # compute_metrics=compute_metrics,
)

## 6. 모델 학습 시작

In [ ]:
# PyTorch CUDA 메모리 할당 정책 변경 (단편화 방지)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 학습 시작 전, GPU 캐시를 한 번 비워줍니다.
torch.cuda.empty_cache()

print("--- LoRA 파인튜닝 시작 ---")
trainer.train()
print("--- 학습 완료 ---")

# 학습 완료 후 최고 성능 모델 저장
trainer.save_model("./results/best_t5_lora_model")

## 7. 공식 테스트셋 예측 (BEA-2019 제출용)

In [ ]:
# 7.1: 공식 테스트 파일(텍스트) 로드
OFFICIAL_TEST_PATH = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/test/ABCN.test.bea19.orig"

if not os.path.exists(OFFICIAL_TEST_PATH):
    print(f"경고: {OFFICIAL_TEST_PATH} 파일을 찾을 수 없습니다. 경로를 확인하세요.")
else:
    print(f"--- 공식 테스트 파일 로드: {OFFICIAL_TEST_PATH} ---")
    official_test_dataset = load_dataset("text", data_files={"test": OFFICIAL_TEST_PATH})["test"]

    # 7.2: 테스트셋을 위한 전처리 함수 (정답[clean]이 없음)
    def preprocess_test_function(examples):
        # "text" 컬럼(원본 문장)에 접두사 추가
        inputs = [PREFIX + doc for doc in examples["text"]] # <--- PREFIX 변수가 이전에 정의되어 있어야 함

        model_inputs = tokenizer(
            inputs,
            max_length=MAX_INPUT_LENGTH, # <--- MAX_INPUT_LENGTH 변수가 이전에 정의되어 있어야 함
            truncation=True,
            padding="max_length"
        )
        return model_inputs

    # 7.3: 테스트셋에 전처리 적용
    print("--- 테스트셋 전처리 시작 ---")
    tokenized_test_set = official_test_dataset.map(
        preprocess_test_function,
        batched=True,
        remove_columns=["text"] # 원본 "text" 컬럼 제거
    )
    print("--- 테스트셋 전처리 완료 ---")

    # 7.4: .predict() 대신 .generate()를 사용한 직접 예측
    print("--- 공식 테스트셋 예측 시작 (model.generate() 사용) ---")

    # ->> 모델과 디바이스 설정
    # (trainer.model은 "load_best_model_at_end=True"에 의해 이미 최고 성능 모델임)
    model = trainer.model
    model.eval() # 평가 모드로 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ->> DataLoader 준비 (배치 처리를 위해)
    tokenized_test_set.set_format(type="torch", columns=["input_ids", "attention_mask"])

    # ->> GPU 메모리에 맞게 조절하세요 (16 or 32)
    PREDICT_BATCH_SIZE = 16
    test_dataloader = DataLoader(tokenized_test_set, batch_size=PREDICT_BATCH_SIZE)

    decoded_preds = [] # 디코딩된 예측을 저장할 리스트

    # ->> 배치 단위로 반복하며 .generate() 호출 (메모리 부족 방지)
    with torch.no_grad(): # 그래디언트 계산 비활성화
        for batch in tqdm(test_dataloader, desc="Generating predictions"):
            # 데이터를 GPU로 이동
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # model.generate() 호출
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=MAX_TARGET_LENGTH  # <--- 훈련 시 사용했던 'MAX_TARGET_LENGTH' 값
                                              #      (혹은 128, 256 등 적절한 최대 길이)
                # num_beams=5, # <--- 필요시 빔 서치 등 옵션 추가
                # early_stopping=True
            )

            # 4. 결과 디코딩 및 저장
            # 생성된 ID를 CPU로 다시 가져와서 디코딩
            batch_preds = tokenizer.batch_decode(generated_ids.cpu(), skip_special_tokens=True)
            decoded_preds.extend(batch_preds)

    # --- 7.5: 예측 결과 디코딩 (위 루프에서 이미 완료됨) ---
    print("--- 예측 완료 ---")

    # --- 7.6: 제출용 파일로 저장 (이 부분은 기존과 동일) ---
    output_filename = "submission.txt"
    with open(output_filename, "w", encoding="utf-8") as f:
        for line in decoded_preds:
            f.write(line.strip() + "\n")

    print(f"--- 결과가 {output_filename} 에 저장되었습니다. ---")
    print("\n--- 예측 샘플 5개 ---")

    # 5개 샘플 출력 (데이터셋 크기가 5보다 작은 경우 대비)
    sample_count = min(5, len(official_test_dataset))
    for i in range(sample_count):
        print(f"Original ({i+1}): {official_test_dataset[i]['text']}") # official_test_dataset은 유지됨
        print(f"Corrected ({i+1}): {decoded_preds[i]}\n")